In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-gemini/genai-processors/blob/main/notebooks/drive_processors.ipynb)

# Drive Processors 🏎

The GenAI Processor library includes a suite of Google Drive processors, which makes it easy to create pipelines grounded in Docs, Sheets &/or Slides.



## Setup 🍳

Run the hidden cell below to install `genai-processors` and import the relevant modules.

In [ ]:
!pip install genai-processors

from genai_processors import processor
from genai_processors.core import drive
from genai_processors.core import genai_model
from genai_processors.core import preamble
from google.colab import auth as colab_auth
from google.colab import userdata
from IPython.display import Markdown, clear_output, display

## Auth 🔐

### Drive Auth

To authenticate with Drive, you can use the `colab_auth` environment in Colab,
or pass a `Credentials` object in other python environments.

### API Key

To use the GenAI model processors, you will need an API key. If you have not
done so already, obtain your API key from Google AI Studio, and import it as a
secret in Colab (recommended) or directly set it below.

In [ ]:
CREDS = None
GOOGLE_API_KEY = None

try:
  colab_auth.authenticate_user()
except Exception:
  print('Failed to authenticate using `colab_auth`.')

try:
  GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except Exception:
  print('Failed to obtain `GOOGLE_API_KEY`.')

## `GoogleSlides` processor 🖼

We can create a pipeline with `GenaiModel` to answer questions about Google Slides.

In [ ]:
PRESENTATION_ID = "YOUR-PRESENTATION-ID"  # @param {type:"string"}
USER_PROMPT = "Describe the presentation in detail."  # @param {type:"string"}

In [ ]:
p_slides = drive.Slides(creds=CREDS)

p_preamble = preamble.Preamble(
    content=f"""You are an expert in reviewing Google Slides presentations.

You have been provided with slides, and must use them to answer the user's question.

User question: {USER_PROMPT}"""
)

p_genai = genai_model.GenaiModel(
    model_name='gemini-2.5-flash', api_key=GOOGLE_API_KEY
)

pipeline = p_slides + p_preamble + p_genai

req = drive.SlidesRequest(presentation_id=PRESENTATION_ID)
req_part = processor.ProcessorPart.from_dataclass(dataclass=req)

input_stream = processor.stream_content([req_part])

streaming_text = ''
async for content_part in pipeline(input_stream):
  streaming_text += content_part.text
  clear_output(wait=True)
  display(Markdown(streaming_text))

## `GoogleDocs` processor 📄

We can create a pipeline with `GenaiModel` to answer questions about a Google Doc.

In [ ]:
DOC_ID = "YOUR-DOC-ID"  # @param {type:"string"}
USER_PROMPT = "Describe the document in detail."  # @param {type:"string"}

In [ ]:
p_docs = drive.Docs(creds=CREDS)

p_preamble = preamble.Preamble(
    content=f"""You are an expert in reviewing Google Docs.

You have been provided with a document, and must use it to answer the user's question.

User question: {USER_PROMPT}"""
)

p_genai = genai_model.GenaiModel(
    model_name='gemini-2.5-flash', api_key=GOOGLE_API_KEY
)

pipeline = p_docs + p_preamble + p_genai

req = drive.DocsRequest(doc_id=DOC_ID)
req_part = processor.ProcessorPart.from_dataclass(dataclass=req)

input_stream = processor.stream_content([req_part])

streaming_text = ''
async for content_part in pipeline(input_stream):
  streaming_text += content_part.text
  clear_output(wait=True)
  display(Markdown(streaming_text))

## `GoogleSheets` processor 🧮

The `GoogleSheets` processor exports sheet data in CSV format, which can be
used to ground a `GenaiModel`.

In [ ]:
SPREADSHEET_ID = "YOUR-SPREADSHEET-ID"  # @param {type:"string"}
USER_PROMPT = "Describe the spreadsheet in granular detail."  # @param {type:"string"}

In [ ]:
p_sheets = drive.Sheets(creds=CREDS)

p_preamble = preamble.Preamble(
    content=f"""You are an expert in reviewing Google Sheets.

You have been provided with a spreadsheet, and must use it to answer the user's question.

User question: {USER_PROMPT}"""
)

p_genai = genai_model.GenaiModel(
    model_name='gemini-2.5-flash', api_key=GOOGLE_API_KEY
)

pipeline = p_sheets + p_preamble + p_genai

req = drive.SheetsRequest(spreadsheet_id=SPREADSHEET_ID)
req_part = processor.ProcessorPart.from_dataclass(dataclass=req)

input_stream = processor.stream_content([req_part])

streaming_text = ''
async for content_part in pipeline(input_stream):
  streaming_text += content_part.text
  clear_output(wait=True)
  display(Markdown(streaming_text))